In [3]:
!pip install fastai


  Using cached fastai-2.8.2-py3-none-any.whl.metadata (9.5 kB)
  Using cached fastdownload-0.0.7-py3-none-any.whl.metadata (5.5 kB)
  Using cached fastcore-1.8.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached fasttransform-0.0.2-py3-none-any.whl.metadata (7.6 kB)
  Using cached fastprogress-1.0.3-py3-none-any.whl.metadata (5.6 kB)
  Using cached plum_dispatch-2.5.7-py3-none-any.whl.metadata (7.5 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.21.1-py3-none-any.whl.metadat

In [4]:
from fastai.vision.all import *
from fastcore.parallel import *
import fastai
import pandas as pd
from pathlib import Path
from pathlib import Path
import torch
from torch.utils.data import Dataset

In [6]:
!pip install timm


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 6.0 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 5.7 MB/s eta 0:00:00


In [16]:
import timm
model = timm.create_model("efficientformerv2_s2", pretrained=True)
torch.save(model.state_dict(), "efficientformerv2_s2_weights.pth")

In [17]:
metadata_path =Path("C:/Users/Yashwanth/isic/train-metadata.csv")
 
hdf5_file=Path('C:/Users/Yashwanth/isic/train-image.hdf5')

In [18]:
df=pd.read_csv(metadata_path)

C:\Users\Yashwanth\AppData\Local\Temp\ipykernel_21720\301825288.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(metadata_path)


In [19]:
import os

for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.endswith(".csv"):
            print(os.path.join(root, file))

In [21]:

new_cat_columns = ['sex', 'anatom_site_general', 'tbp_lv_location', 'tbp_lv_location_simple']


In [22]:
# Drop specified columns
columns_to_drop = ['copyright_license', 'attribution', 'image_type', 'iddx_1', 'iddx_2', 'iddx_3', 'iddx_4',
                   'iddx_5', 'iddx_full', 'mel_mitotic_index', 'mel_thick_mm', 'tbp_tile_type', 
                   'tbp_lv_dnn_lesion_confidence', 'lesion_id']

# Define categorical columns manually (based on what you likely intended)
cat_names = ['sex', 'anatom_site_general', 'tbp_lv_location', 'tbp_lv_location_simple']

# Manually set new_cat_columns (fallback because CSV is missing)
new_cat_columns = cat_names  # You can edit or expand this list later if needed

# Define continuous columns (all columns not in categorical or dropped list)
cont_names = [x for x in df.columns if x not in (cat_names + ['target', 'isic_id', 'patient_id'] + columns_to_drop)]

# Define the target and image ID columns
y_col = 'target'
image_col = 'isic_id'


# Create a custom dataset that includes both image and tabular data: